##  Decorators

In [1]:
def decorate1(func):
    def inner():
        print("apply decorate1")
        r = func() #func is a non local variable
        return r

    return inner


@decorate1
def hello():
    """doc di hello"""
    print("hello, world!")


@decorate1
def bye():
    print("bye bye")


hello()
bye()

apply decorate1
hello, world!
apply decorate1
bye bye


In [2]:
hello?? #we loose the source 

Signature: hello()
Docstring: <no docstring>
Source:   
    def inner():
        print("apply decorate1")
        r = func() #func is a non local variable
        return r
File:      ~/Desktop/Corsi/Ad_Progr/advanced_programming_2021/python/02_symbols/<ipython-input-1-0dd4d9f241f5>
Type:      function


In [3]:
def decorate1(func):
    def inner():
        print("decorate 1")
        r = func()
        # ...
        return r

    return inner


def hello():
    print("hello, world!")
    return "pippo"


hello = decorate1(hello)  # what @decorate1 does

a = hello()  # executes inner
print(a)

decorate 1
hello, world!
pippo


In [4]:
help(hello)

Help on function inner in module __main__:

inner()



In [5]:
def decorate2(func):
    def inner():
        print("apply decorate2")
        r = func()
        return r

    return inner


@decorate1
@decorate2
def hello12():
    print("hello, world!")


hello12()  # same as hello12 = decorate1(decorate2(hello12))

decorate 1
apply decorate2
hello, world!


In [6]:
@decorate2
@decorate1
def hello21():
    print("hello, world!")


hello21()  # same as hello21 = decorate2(decorate1(hello21))

apply decorate2
decorate 1
hello, world!


### Remember to use `@functools.wraps` in order to have the right source in the ??

In [7]:
import functools


def decorate1(func):
    @functools.wraps(func)
    def inner():
        print("apply decorate1")
        func()

    return inner


@decorate1
def hello():
    "doc for hello"
    print("hello, world!")


help(hello)

Help on function hello in module __main__:

hello()
    doc for hello



In [8]:
hello?? #no more inner

Signature: hello()
Source:   
@decorate1
def hello():
    "doc for hello"
    print("hello, world!")
File:      ~/Desktop/Corsi/Ad_Progr/advanced_programming_2021/python/02_symbols/<ipython-input-7-375346aac97e>
Type:      function


In [9]:
hello.__doc__ = "pippo" #you can overwrite it

In [10]:
help(hello)

Help on function hello in module __main__:

hello()
    pippo



###  How to pass arguments to the inner function

In [11]:
# adapted from Fluent Python
import functools


def args_to_string(*args, **kw):
    arg_str = ()
    if args:
        arg_str += ((",".join(str(arg) for arg in args)),)
    if kw:
        arg_str += ((", ".join(("{0}={1}".format(k, v) for k, v in kw.items()))),)
    return ",".join(a for a in arg_str)

In [17]:
from time import perf_counter, sleep


def time_this(func):
    @functools.wraps(func)
    def decorated(*args, **kw): #it takes whatever number of keywords and positional arguments
        t0 = perf_counter()
        result = func(*args, **kw)
        t1 = perf_counter()
        name = func.__name__
        arg_str = args_to_string(*args, **kw)
        # print('{}({}): [{:0.8f}]'.format(name, arg_str,t1-t0))
        # print('%s(%s): [%0.8f s]' % (name, arg_str, t1-t0))
        print(f"{name}({arg_str}):[{t1-t0:0.8f} s]")
        return result

    return decorated


@time_this
def wait(seconds):
    sleep(seconds)


@functools.lru_cache(4)  # <-- note () # parametrized decorators: usign the default values otherwise:
#If *maxsize* is set to None, the LRU features are disabled and the cache can grow without bound.
#stores a dictionary in which the keys are the n and the value is the valuebb
@time_this
def factorial(n):
    return 1 if n < 2 else n * factorial(n - 1)


@time_this
def sum(a, b):
    return a + b


@time_this
def dummy(*args, **kw):
    a = args
    b = kw


wait(0.3)
factorial(10)
sum(4, 5)
dummy("pos", "second", a="a", b="b")

wait(0.3):[0.30358614 s]
factorial(1):[0.00000038 s]
factorial(2):[0.00004458 s]
factorial(3):[0.00007205 s]
factorial(4):[0.00010214 s]
factorial(5):[0.00011938 s]
factorial(6):[0.00013451 s]
factorial(7):[0.00015016 s]
factorial(8):[0.00016422 s]
factorial(9):[0.00017828 s]
factorial(10):[0.00019388 s]
sum(4,5):[0.00000093 s]
dummy(pos,second,a=a, b=b):[0.00000121 s]


In [14]:
factorial(6) #already computed: just returns the value of the factorial

720

In [15]:
factorial(12)# not computed yet!

factorial(11):[0.00000174 s]
factorial(12):[0.00007635 s]


479001600

In [19]:
functools.lru_cache?

Signature: functools.lru_cache(maxsize=128, typed=False)
Docstring:
Least-recently-used cache decorator.

If *maxsize* is set to None, the LRU features are disabled and the cache
can grow without bound.

If *typed* is True, arguments of different types will be cached separately.
For example, f(3.0) and f(3) will be treated as distinct calls with
distinct results.

Arguments to the cached function must be hashable.

View the cache statistics named tuple (hits, misses, maxsize, currsize)
with f.cache_info().  Clear the cache and statistics with f.cache_clear().
Access the underlying function with f.__wrapped__.

See:  http://en.wikipedia.org/wiki/Cache_replacement_policies#Least_recently_used_(LRU)
File:      ~/opt/anaconda3/lib/python3.8/functools.py
Type:      function


In [16]:
dir(factorial)

['__annotations__',
 '__call__',
 '__class__',
 '__copy__',
 '__deepcopy__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__get__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__name__',
 '__ne__',
 '__new__',
 '__qualname__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__wrapped__',
 'cache_clear',
 'cache_info']

In [18]:
def parametrized_time_this(check=True):
    def decorator(func):
        if not check:
            return func

        @functools.wraps(func)
        def decorated(*args, **kw):
            t0 = perf_counter()
            result = func(*args, **kw)
            t1 = perf_counter()
            name = func.__name__
            arg_str = args_to_string(*args, **kw)
            print(f"{name}({arg_str}):[{t1-t0:0.8f} s]")
            return result

        return decorated

    return decorator  # <-- returns the actual decorator


debug = True


@parametrized_time_this(debug)
def wait(seconds):
    print("going to sleep for", seconds, "seconds")
    sleep(seconds)
    print("woke up!")


wait(0.4)

going to sleep for 0.4 seconds
woke up!
wait(0.4):[0.40243869 s]


### Decorators as function objects

In [20]:
class TimeThis: #implement decorator as a class
    def __init__(self, func):  # <--
        self._func = func  # <--
        functools.update_wrapper(self, func)  # <--

    def __call__(self, *args, **kw):
        t0 = perf_counter()
        result = self._func(*args, **kw)  # <--
        t1 = perf_counter()
        name = self._func.__name__  # <--
        arg_str = args_to_string(*args, **kw)
        print(f"{name}({arg_str}):[{t1-t0:0.8f} s]")

        return result


@TimeThis
def wait(seconds):
    "doc"
    print("going to sleep for", seconds, "seconds")
    sleep(seconds)
    print("woke up!")

#wait = TimeThis(wait) : it's the equivalent! It's an object of the class TimeThis
wait(0.4)

going to sleep for 0.4 seconds
woke up!
wait(0.4):[0.40074306 s]


In [21]:
wait??

Signature:      wait(seconds)
Call signature: wait(*args, **kw)
Type:           TimeThis
String form:    <__main__.TimeThis object at 0x7f816d3ad760>
File:           ~/Desktop/Corsi/Ad_Progr/advanced_programming_2021/python/02_symbols/<ipython-input-20-00c190c75f36>
Source:        
@TimeThis
def wait(seconds):
    "doc"
    print("going to sleep for", seconds, "seconds")
    sleep(seconds)
    print("woke up!")


In [22]:
class ParametrizedTimeThis:
    def __init__(self, check=True):
        self.check = check

    def __call__(self, func): #the call operators returns the inner func
        if self.check:
            # return TimeThis(func)

            @functools.wraps(func)
            @TimeThis
            def wrapper(*args, **kwargs):
                return func(*args, **kwargs)

            return wrapper
        return func


@ParametrizedTimeThis(True)
def wait(seconds):
    print("going to sleep for", seconds, "seconds")
    sleep(seconds)
    print("woke up!")


wait(0.4)

going to sleep for 0.4 seconds
woke up!
wrapper(0.4):[0.40557925 s]


In [23]:
wait??

Signature:      wait(seconds)
Call signature: wait(*args, **kw)
Type:           TimeThis
String form:    <__main__.TimeThis object at 0x7f816d39e550>
File:           ~/Desktop/Corsi/Ad_Progr/advanced_programming_2021/python/02_symbols/<ipython-input-22-eb4a5286fb36>
Source:        
@ParametrizedTimeThis(True)
def wait(seconds):
    print("going to sleep for", seconds, "seconds")
    sleep(seconds)
    print("woke up!")


In [24]:
PTT = ParametrizedTimeThis(True) #an alias: a sticky notes as p = Point


@PTT
def dummy(*args, **kw):
    pass


dummy(0.4)

wrapper(0.4):[0.00000118 s]


In [25]:
dummy??

Signature:   dummy(*args, **kw)
Type:        TimeThis
String form: <__main__.TimeThis object at 0x7f816d3ada00>
File:        ~/Desktop/Corsi/Ad_Progr/advanced_programming_2021/python/02_symbols/<ipython-input-24-031b87aad118>
Source:     
@PTT
def dummy(*args, **kw):
    pass
